# jpegio 사용법
### (Last updated 2019.03.19)
##### *如果您发现错误或有需要的功能，请随时通知我们（leedaewon@nsr.re.kr、daewon4you@gmail.com）*



- `jpegio' 是一个 Python 包，它通过将 [libjpeg] （https://www.ijg.org） 中的某些 JPEG I/O 功能创建为 Python 模块（i.e.，wrapping）API，该模块在 C 语言中实现.
- 参考了希腊ITI-CERTH研究所[MKLab]（https://mklab.iti.gr）提供的[https://github.com/MKLab-ITI/image-forensics）源代码.
- Uber research也提供 [类似代码] （https://github.com/uber-research/jpeg2dct）
- 使用 [Cython] （https://cython.org/） 从 C 代码生成 Python 模块.
- 微软 Windows 使用 [libjpeg-turbo]（https://libjpeg-turbo.org）.
- UNIX 系列操作系统包括编译“libjpeg”源代码的过程，包括安装“jpegio”包的过程.




In [1]:
import jpegio as jio

#### 检索 JPEG 图像
- 默认情况下，解压缩的 JPEG 数据通过“解压缩Jpeg”对象进行处理.
- 您还可以使用其他对象（e.g.，“ZigzagDct1d”）将 DCT 系数视为单独的数据结构（后面将详细介绍）).

In [2]:
fpath = "../tests/images/cherries01.jpg"  # JPEG 文件地址
jpeg = jio.read(fpath)
type(jpeg)

jpegio.decompressedjpeg.DecompressedJpeg

#### 检查图像大小

In [3]:
jpeg.image_width, jpeg.image_width

(756, 756)

#### 检查 YCbCr 通道数

In [4]:
jpeg.num_components

3

#### 访问 DCT 系数
- 成员变量名为“coef_arrays”，是 Python 的默认列表对象.
- “coef_arrays”列表包含每个通道对应的 DCT 系数数组.
- 每个 DCT 系数数组都是二维“numpy.ndarray”对象.
- 不将 DCT 系数数组作为三维“numpy.ndarray”进行管理的原因是，DCT 系数数组的大小可能因通道而异.
- 如果 DCT 系数数组的大小因通道而异，则在 JPEG 压缩过程中对 CbCr 通道应用了 down sampling。

In [5]:
type(jpeg.coef_arrays) 

list

In [6]:
type(jpeg.coef_arrays[0])

numpy.ndarray

In [7]:
len(jpeg.coef_arrays)  # 等于通道数

3

In [8]:
print(jpeg.coef_arrays[0].ndim)  # 第一个 DCT 系数数组中的维度
print(jpeg.coef_arrays[1].ndim)  # 第二个 DCT 系数数组中的维度
print(jpeg.coef_arrays[2].ndim)  # 第三个 DCT 系数数组的维度

2
2
2


In [9]:
# 每个通道仅输出第一个 DCT 计数块
for i in range(jpeg.num_components):
    coef = jpeg.coef_arrays[i]
    print("[Channel #%d] The 1st DCT coef. block"%(i+1))
    print(coef[:8, :8])

[Channel #1] The 1st DCT coef. block
[[-567   11  -47   -6    4    2    1    1]
 [ -81  -41   22   13   -5   -2   -2   -4]
 [  19   15   10  -12    3   -1    1    1]
 [   8   -7   -7   -4    3   -3    0   -2]
 [  -7   -5    6    3   -2   -2   -2   -2]
 [  10    0  -10   -5   -3    1   -1   -1]
 [  -2   -4   -1    2    2   -2   -2    1]
 [  -3    0    4   -3   -3    0   -1   -1]]
[Channel #2] The 1st DCT coef. block
[[-58   6   1   8   0   0   2   0]
 [ 11   5  -4   3   6  -3   1  -2]
 [  5  -7  -2   1  -2  -2  -2   0]
 [ -3  -3   5  -1  -3   1   2   2]
 [  3   0  -6  -2   2   1  -1   0]
 [  4  -1   2   1  -1  -3  -2  -1]
 [ -2   0   2   0   0  -1   0   0]
 [  2   0  -2  -1   1   2   1  -1]]
[Channel #3] The 1st DCT coef. block
[[-4 -4 -3 -2  0  1  2 -1]
 [-2 -2 -3  3  0  1 -2  0]
 [-9  9  2 -1  3 -1  0  0]
 [ 4  1 -2 -5 -2  0  0  0]
 [-3 -2 -2 -1  0 -1  1  0]
 [-2  1  1 -1  1  1  0  0]
 [ 1 -2 -1 -1  0  1  0 -1]
 [ 0 -2  0  0  0 -1  0  0]]


In [10]:
# 每个通道的第一个 DCT 系数数组的大小输出
# （您会注意到 DCT 数组的大小不同）
for i in range(jpeg.num_components):
    coef = jpeg.coef_arrays[i]
    print("[Channel #%d] Size of DCT coef. array: %s"%(i+1, coef.shape))

[Channel #1] Size of DCT coef. array: (504, 760)
[Channel #2] Size of DCT coef. array: (256, 384)
[Channel #3] Size of DCT coef. array: (256, 384)


#### DCT 系数“numpy.ndarray”阵列形状变形
- 为了更有效地使用 DCT 系数数组，需要修改数组形状。
- 例如，如果要逐块处理索引，则更易于使用索引（块行、块列、8x8 排列行、8x8 排列列）.
- 正确使用“numpy.reshape”和“numpy.transpose”.
-“numpy.reshape”和“numpy.transpose”只更改数据视图，而不更改内部内存结构，因此无需担心性能问题。.

In [11]:
# 当您希望以 8x8 块为单位访问数组时，
# 如下图所示，可以改变数组的外观.

coef = jpeg.coef_arrays[0]  # 第一个通道 DCT 系数数组
nr_blk = coef.shape[0] // 8  # 8x8 块单位行数
nc_blk = coef.shape[1] // 8  # 8x8 块单位列数
print(nr_blk, nc_blk)

63 95


In [12]:
# 以八个单位切割十个块（nc_blk）,
# 再次将块列切为 8 个块行数（nr_blk）.

coef_blk = coef.reshape(nr_blk, 8, nc_blk, 8)
print(coef_blk.shape)

(63, 8, 95, 8)


In [13]:
# 重新定位阵列的轴，使索引更易于使用.
# 无需考虑数据在内部的存储方式,
#您只需要将索引视为所需（反正内部数据存储器是一维数组）

coef_blk = coef_blk.transpose(0, 2, 1, 3)
print(coef_blk.shape)

(63, 95, 8, 8)


In [14]:
# 第 3 行第 2 列中的 DCT 系数块
coef_blk[3, 2, :, :]

array([[-644,   -6,    3,  -11,    2,   -2,    7,   -1],
       [ -26,  -27,   28,    5,   -5,    0,   -4,    0],
       [  -4,   -6,    7,   15,   -1,   -2,   -4,    3],
       [   2,   17,  -17,    0,   -1,    1,    1,    0],
       [   0,   -8,    5,    0,    4,   -2,   -2,    1],
       [  -5,   -8,    6,    4,   -1,    0,   -3,   -2],
       [   1,    2,   -3,   -2,   -1,    0,    3,    0],
       [   2,    1,    4,   -1,    0,   -2,   -1,   -1]], dtype=int16)

In [15]:
# 第 10 行第 10 列中的 DCT 系数块
coef_blk[10, 10, :, :]

array([[-674,    0,  -51,   -3,   16,   -3,    0,   -3],
       [ -61,   15,  -58,   14,   15,   10,   -4,    2],
       [  -4,  -11,  -13,    2,   15,    4,    3,    1],
       [   8,   -2,  -13,   -3,    5,   -2,   -2,    1],
       [   2,    2,   -2,   -7,    3,   -5,   -2,    0],
       [ -16,    6,    8,    1,   -7,   -5,    3,    0],
       [  -2,   -1,    8,    1,   -9,   -4,    0,    1],
       [   4,    0,   -5,   -3,    5,   -1,    5,    0]], dtype=int16)

In [16]:
# 只需将块索引放在下面即可.
coef_blk[3, 4]

array([[-680,  -29,    7,   -8,   -2,   -3,   -1,    4],
       [ -19,  -28,    6,   12,   -4,    3,    0,    2],
       [  -5,  -14,    0,    9,    1,   -2,   -1,    1],
       [   0,    8,    6,   -5,   -1,   -4,    0,    4],
       [  -2,   -6,    1,    3,   -5,    0,    0,   -2],
       [  -8,   -5,    0,    5,    6,   -3,    2,    0],
       [   1,   -1,    0,    1,   -1,    0,    1,    2],
       [   0,    3,    1,   -3,   -3,   -1,    0,    0]], dtype=int16)

#### 按渠道查看信息
- 要查看 JPEG 频道特定信息，请使用成员变量“decompressedJpeg”中的“comp_info”.
- 包含与 Downsampling 相关的各种信息，尤其是在处理在 CbCr 通道中应用多环的 JPEG 时，可以检查大小信息。.
- 例如，“通信信息”对象的“v_samp_factor”和“h_samp_factor”是 YCbCr 中每个通道的 downsampling 比率。如果您只需要在 downsampling 后创建图像大小，则可以使用“downsampled_width”和“downsampled_height”对象。.

In [17]:
# comp_info列表对象，包含特定于每个通道的客户端信息对象.
# "component"对应于 "channel".

type(jpeg.comp_info)

list

In [18]:
type(jpeg.comp_info[0])  # ComponentInfo 对象

jpegio.componentinfo.ComponentInfo

In [19]:
jpeg.comp_info[0]  # 第一个channel的ComponentInfo

In [20]:
for ci in jpeg.comp_info:
    print("[Component #%d]"%(ci.component_id))
    print("Quantization table number:", ci.quant_tbl_no)
    print("DC table number:", ci.dc_tbl_no)
    print("AC table number:", ci.ac_tbl_no)
    print("Width after downsampling:", ci.downsampled_width)  # 缩减采样后的图像水平大小
    print("Height after downsampling:", ci.downsampled_height)  # 缩减像素采样后的图像垂直大小
    print("Width in blocks:", ci.width_in_blocks)  # 块行数
    print("Height in blocks:", ci.height_in_blocks)  # 10 个块
    print("Vertical sampling factor:", ci.h_samp_factor)  # 行采样因子
    print("Horizontal sampling factor:", ci.v_samp_factor)  # 热采样因子
    print()

[Component #1]
Quantization table number: 0
DC table number: 0
AC table number: 0
Width after downsampling: 756
Height after downsampling: 504
Width in blocks: 95
Height in blocks: 63
Vertical sampling factor: 2
Horizontal sampling factor: 2

[Component #2]
Quantization table number: 1
DC table number: 1
AC table number: 1
Width after downsampling: 378
Height after downsampling: 252
Width in blocks: 48
Height in blocks: 32
Vertical sampling factor: 1
Horizontal sampling factor: 1

[Component #3]
Quantization table number: 1
DC table number: 1
AC table number: 1
Width after downsampling: 378
Height after downsampling: 252
Width in blocks: 48
Height in blocks: 32
Vertical sampling factor: 1
Horizontal sampling factor: 1



#### 非零 DCT 交流系数（非零 DCT 交流系数）的数量  
- 在 8x8 DCT 系数块中，第一个系数（第 0 行 0 列）称为直流系数，其余系数称为交流系数.
- 由于大多数触及 JPEG DCT 系数的固加诺格拉菲工具都针对交流系数，因此需要获取除直流系数之外的交流系数数.
- “jpegio”提供名为“count_nnz_ac”的成员标签。“count_nnz_ac”告诉您所有 DCT 系数块中非零交流系数的数量。也就是说，除直流系数外，从其余系数中求出非零系数的数量.

In [21]:
jpeg.count_nnz_ac()

476659

- 如果想要为每个通道获取非零交流系数的数量，请使用下面的代码。

In [22]:
import numpy as np

for i in range(jpeg.num_components):
    coef = jpeg.coef_arrays[i]
    nnz_total = np.count_nonzero(coef)  # 所有 DCT 系数中的非零系数数
    nnz_dc = np.count_nonzero(coef[::8, ::8])  # 非零直流系数数
    print("[Channel #%d] Number of non-zero DCT AC coefficients: %d"%(i+1, nnz_total - nnz_dc))

[Channel #1] Number of non-zero DCT AC coefficients: 327921
[Channel #2] Number of non-zero DCT AC coefficients: 76925
[Channel #3] Number of non-zero DCT AC coefficients: 71813


#### 通过 Zig-Zag 扫描将 DCT 系数读入一维数组
- 根据需要，可能需要以 zig-zag 扫描方式检索的 DCT 系数的一维数组。.
- 在 Python 中逐块进行 zig-zag 扫描处理可能会降低性能.
- jpegio 提供“ZigzagDct1d”类，这是“DecompressedJpeg”的子类。.
-要将 JPEG 读为“ZigzagDct1d”对象，必须指定一个标志.

In [23]:
# 顺便说一下，DecompressedJpeg 是 jpegio.被指定为DECOMPRESSED.
jpeg_zz = jio.read(fpath, jio.ZIGZAG_DCT_1D)

In [24]:
type(jpeg_zz)

jpegio.zigzagdctjpeg.ZigzagDct1d

In [25]:
coef = jpeg_zz.coef_arrays[0]
coef.shape

(63, 95, 64)

- 可以看出，DCT 系数数组中最后一个维度的大小为 64（一维数组大小，而不是 8x8 数组的二维）.
- 以下是 Python 代码与 zig-zag 扫描性能的比较.

In [26]:
import os
import glob
import time

BS = 8  # Size of the DCT square block width

list_fpaths = []

for fpath in glob.glob(os.path.join('../tests/images', '*.jpg')):
    list_fpaths.append(fpath)
        
for fpath in list_fpaths:
        # Read DCT with ZigzagDct1d
        time_beg_zz = time.time()
        jpeg_zz = jio.read(fpath, jio.ZIGZAG_DCT_1D)
        list_coef_zz = []
        for c in range(jpeg_zz.num_components):                
            nrows_blk, ncols_blk = jpeg_zz.get_coef_block_array_shape(c)

            arr_zz = jpeg_zz.coef_arrays[c].reshape(nrows_blk*ncols_blk, BS*BS)
            list_coef_zz.append(arr_zz)
        # end of for
        time_elapsed_zz = time.time() - time_beg_zz

        # Read DCT with DecompressedJpeg
        time_beg_de = time.time()
        jpeg_de = jio.read(fpath, jio.DECOMPRESSED)
        list_coef_de = []
        for c in range(jpeg_de.num_components):
            arr_de = jpeg_de.coef_arrays[c]
            nrows_blk, ncols_blk = jpeg_de.get_coef_block_array_shape(c)
            arr_de = arr_de.reshape(nrows_blk, BS, ncols_blk, BS)
            arr_de = arr_de.transpose(0, 2, 1, 3)
            arr_de = arr_de.reshape(nrows_blk, ncols_blk, BS, BS)

            zz_de = np.zeros((nrows_blk, ncols_blk, BS*BS),
                             dtype=np.int16)

            # Zigzag scanning over DCT blocks.
            for i in range(nrows_blk):
                for j in range(ncols_blk):
                    zz_de[i, j][0] = arr_de[i, j][0, 0]

                    zz_de[i, j][1] = arr_de[i, j][0, 1]
                    zz_de[i, j][2] = arr_de[i, j][1, 0]

                    zz_de[i, j][3] = arr_de[i, j][2, 0]
                    zz_de[i, j][4] = arr_de[i, j][1, 1]
                    zz_de[i, j][5] = arr_de[i, j][0, 2]

                    zz_de[i, j][6] = arr_de[i, j][0, 3]
                    zz_de[i, j][7] = arr_de[i, j][1, 2]
                    zz_de[i, j][8] = arr_de[i, j][2, 1]
                    zz_de[i, j][9] = arr_de[i, j][3, 0]

                    zz_de[i, j][10] = arr_de[i, j][4, 0]
                    zz_de[i, j][11] = arr_de[i, j][3, 1]
                    zz_de[i, j][12] = arr_de[i, j][2, 2]
                    zz_de[i, j][13] = arr_de[i, j][1, 3]
                    zz_de[i, j][14] = arr_de[i, j][0, 4]

                    zz_de[i, j][15] = arr_de[i, j][0, 5]
                    zz_de[i, j][16] = arr_de[i, j][1, 4]
                    zz_de[i, j][17] = arr_de[i, j][2, 3]
                    zz_de[i, j][18] = arr_de[i, j][3, 2]
                    zz_de[i, j][19] = arr_de[i, j][4, 1]
                    zz_de[i, j][20] = arr_de[i, j][5, 0]

                    zz_de[i, j][21] = arr_de[i, j][6, 0]
                    zz_de[i, j][22] = arr_de[i, j][5, 1]
                    zz_de[i, j][23] = arr_de[i, j][4, 2]
                    zz_de[i, j][24] = arr_de[i, j][3, 3]
                    zz_de[i, j][25] = arr_de[i, j][2, 4]
                    zz_de[i, j][26] = arr_de[i, j][1, 5]
                    zz_de[i, j][27] = arr_de[i, j][0, 6]

                    zz_de[i, j][28] = arr_de[i, j][0, 7]
                    zz_de[i, j][29] = arr_de[i, j][1, 6]
                    zz_de[i, j][30] = arr_de[i, j][2, 5]
                    zz_de[i, j][31] = arr_de[i, j][3, 4]
                    zz_de[i, j][32] = arr_de[i, j][4, 3]
                    zz_de[i, j][33] = arr_de[i, j][5, 2]
                    zz_de[i, j][34] = arr_de[i, j][6, 1]
                    zz_de[i, j][35] = arr_de[i, j][7, 0]

                    zz_de[i, j][36] = arr_de[i, j][7, 1]
                    zz_de[i, j][37] = arr_de[i, j][6, 2]
                    zz_de[i, j][38] = arr_de[i, j][5, 3]
                    zz_de[i, j][39] = arr_de[i, j][4, 4]
                    zz_de[i, j][40] = arr_de[i, j][3, 5]
                    zz_de[i, j][41] = arr_de[i, j][2, 6]
                    zz_de[i, j][42] = arr_de[i, j][1, 7]

                    zz_de[i, j][43] = arr_de[i, j][2, 7]
                    zz_de[i, j][44] = arr_de[i, j][3, 6]
                    zz_de[i, j][45] = arr_de[i, j][4, 5]
                    zz_de[i, j][46] = arr_de[i, j][5, 4]
                    zz_de[i, j][47] = arr_de[i, j][6, 3]
                    zz_de[i, j][48] = arr_de[i, j][7, 2]

                    zz_de[i, j][49] = arr_de[i, j][7, 3]
                    zz_de[i, j][50] = arr_de[i, j][6, 4]
                    zz_de[i, j][51] = arr_de[i, j][5, 5]
                    zz_de[i, j][52] = arr_de[i, j][4, 6]
                    zz_de[i, j][53] = arr_de[i, j][3, 7]

                    zz_de[i, j][54] = arr_de[i, j][4, 7]
                    zz_de[i, j][55] = arr_de[i, j][5, 6]
                    zz_de[i, j][56] = arr_de[i, j][6, 5]
                    zz_de[i, j][57] = arr_de[i, j][7, 4]

                    zz_de[i, j][58] = arr_de[i, j][7, 5]
                    zz_de[i, j][59] = arr_de[i, j][6, 6]
                    zz_de[i, j][60] = arr_de[i, j][5, 7]

                    zz_de[i, j][61] = arr_de[i, j][6, 7]
                    zz_de[i, j][62] = arr_de[i, j][7, 6]

                    zz_de[i, j][63] = arr_de[i, j][7, 7]
                # end of for (j)
            # end of for (i)
            list_coef_de.append(zz_de)
        # end of for (c)
        time_elapsed_de = time.time() - time_beg_de
        print("[File: %s]"%(os.path.basename(fpath)))
        print("[Time] C-optimized: %f, Naive Python: %f" \
              %(time_elapsed_zz, time_elapsed_de), end='\n\n')

[File: arborgreens01.jpg]
[Time] C-optimized: 0.008000, Naive Python: 0.441206

[File: arborgreens02.jpg]
[Time] C-optimized: 0.010014, Naive Python: 0.406220

[File: arborgreens03.jpg]
[Time] C-optimized: 0.012001, Naive Python: 0.442000

[File: arborgreens04.jpg]
[Time] C-optimized: 0.009000, Naive Python: 0.423001

[File: arborgreens05.jpg]
[Time] C-optimized: 0.009999, Naive Python: 0.453000

[File: arborgreens06.jpg]
[Time] C-optimized: 0.011000, Naive Python: 0.484000

[File: arborgreens07.jpg]
[Time] C-optimized: 0.009002, Naive Python: 0.483999

[File: arborgreens08.jpg]
[Time] C-optimized: 0.010000, Naive Python: 0.477998

[File: arborgreens09.jpg]
[Time] C-optimized: 0.010000, Naive Python: 0.483999

[File: arborgreens10.jpg]
[Time] C-optimized: 0.009001, Naive Python: 0.490000

[File: cherries01.jpg]
[Time] C-optimized: 0.008000, Naive Python: 0.468999

[File: cherries02.jpg]
[Time] C-optimized: 0.011000, Naive Python: 0.476000

[File: cherries03.jpg]
[Time] C-optimized: 0.0